In [1]:
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

/opt/anaconda3/envs/envallen/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_dir = "./allendata"
manifest_path = os.path.join(data_dir, "manifest.json")
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [3]:
sessions = cache.get_session_table() #Returns a Pandas dataframe
sessions.head() #Sneak peek of the table

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


In [4]:
units = cache.get_units()
units.head()

,waveform_PT_ratio,waveform_amplitude,amplitude_cutoff,cumulative_drift,d_prime,waveform_duration,ecephys_channel_id,firing_rate,waveform_halfwidth,isi_violations,...,phase,sampling_rate,has_lfp_data,date_of_acquisition,published_at,specimen_id,session_type,age_in_days,sex,genotype
id,,,,,,,,,,,,,,,,,,,,,
915956282,0.611816,164.878740,0.072728,309.71,3.910873,0.535678,850229419,6.519432,0.164824,0.104910,...,3a,29999.915391,True,2019-01-09T00:26:20Z,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt
915956340,0.439372,247.254345,0.000881,160.24,5.519024,0.563149,850229419,9.660554,0.206030,0.006825,...,3a,29999.915391,True,2019-01-09T00:26:20Z,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt
915956345,0.500520,251.275830,0.001703,129.36,3.559911,0.521943,850229419,12.698430,0.192295,0.044936,...,3a,29999.915391,True,2019-01-09T00:26:20Z,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt
915956349,0.424620,177.115380,0.096378,169.29,2.973959,0.508208,850229419,16.192413,0.192295,0.120715,...,3a,29999.915391,True,2019-01-09T00:26:20Z,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt
915956356,0.512847,214.954545,0.054706,263.01,2.936851,0.549414,850229419,2.193113,0.233501,0.430427,...,3a,29999.915391,True,2019-01-09T00:26:20Z,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt


In [5]:
session = 799864342

In [6]:
oursession = cache.get_session_data(session) #Returns a session object

Downloading: 100%|██████████| 2.74G/2.74G [08:06<00:00, 5.63MB/s] 
/opt/anaconda3/envs/envallen/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/anaconda3/envs/envallen/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.2.2 because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


In [7]:
oursession.units.head()

/opt/anaconda3/envs/envallen/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/anaconda3/envs/envallen/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.2.2 because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/anaconda3/envs/envallen/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/anaconda3/envs/envallen/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.2.2 because vers

,waveform_PT_ratio,waveform_amplitude,amplitude_cutoff,cluster_id,cumulative_drift,d_prime,firing_rate,isi_violations,isolation_distance,L_ratio,...,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,
951087020,0.419909,56.968080,0.038877,7,124.89,3.828933,8.672372,0.040607,178.889574,0.055540,...,382.0,CA1,7759.0,1658.0,7371.0,probeA,See electrode locations,29999.960987,1249.998374,True
951087105,0.305510,56.118465,0.027375,17,313.13,6.138869,3.532412,0.165985,75.868315,0.002614,...,382.0,CA1,7705.0,1541.0,7420.0,probeA,See electrode locations,29999.960987,1249.998374,True
951087211,0.305828,145.770105,0.002336,29,171.47,3.647205,1.448798,0.267586,38.362057,0.024326,...,382.0,CA1,7672.0,1470.0,7454.0,probeA,See electrode locations,29999.960987,1249.998374,True
951087291,0.301195,161.778435,0.013721,40,140.25,2.609966,4.083122,0.248460,33.501676,0.066943,...,382.0,CA1,7658.0,1440.0,7469.0,probeA,See electrode locations,29999.960987,1249.998374,True
951087261,0.308870,150.684300,0.000681,36,160.88,3.178074,2.275519,0.277960,37.621567,0.050031,...,382.0,CA1,7658.0,1440.0,7469.0,probeA,See electrode locations,29999.960987,1249.998374,True


In [8]:
oursession.metadata['stimulus_names']

/opt/anaconda3/envs/envallen/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/anaconda3/envs/envallen/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.2.2 because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/anaconda3/envs/envallen/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/anaconda3/envs/envallen/lib/python3.11/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.2.2 because vers

['spontaneous',
 'gabors',
 'flashes',
 'drifting_gratings',
 'natural_movie_three',
 'natural_movie_one',
 'static_gratings',
 'natural_scenes',
 'drifting_gratings_contrast']

In [9]:
oursession.get_stimulus_table().head()

,stimulus_block,start_time,stop_time,x_position,temporal_frequency,frame,spatial_frequency,size,contrast,stimulus_name,phase,color,orientation,y_position,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,,,,,,,
0,null,29.005689,89.072529,null,null,null,null,null,null,spontaneous,null,null,null,null,60.066840,0
1,0.0,89.072529,89.306050,-10.0,4.0,null,0.08,"[20.0, 20.0]",0.8,gabors,"[3644.93333333, 3644.93333333]",null,90.0,20.0,0.233521,1
2,0.0,89.306050,89.556250,20.0,4.0,null,0.08,"[20.0, 20.0]",0.8,gabors,"[3644.93333333, 3644.93333333]",null,45.0,-10.0,0.250201,2
3,0.0,89.556250,89.806451,-40.0,4.0,null,0.08,"[20.0, 20.0]",0.8,gabors,"[3644.93333333, 3644.93333333]",null,45.0,-20.0,0.250201,3
4,0.0,89.806451,90.056651,10.0,4.0,null,0.08,"[20.0, 20.0]",0.8,gabors,"[3644.93333333, 3644.93333333]",null,0.0,0.0,0.250201,4


In [10]:
stimuli_table = oursession.get_stimulus_table('drifting_gratings')
stimuli_table.head()

,stimulus_block,start_time,stop_time,temporal_frequency,spatial_frequency,size,contrast,stimulus_name,phase,orientation,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,,,
3798,2.0,1590.310709,1592.312419,4.0,0.04,"[250.0, 250.0]",0.8,drifting_gratings,"[42471.86666667, 42471.86666667]",135.0,2.00171,246
3799,2.0,1593.313231,1595.314911,15.0,0.04,"[250.0, 250.0]",0.8,drifting_gratings,"[42471.86666667, 42471.86666667]",135.0,2.00168,247
3800,2.0,1596.315739,1598.317439,15.0,0.04,"[250.0, 250.0]",0.8,drifting_gratings,"[42471.86666667, 42471.86666667]",45.0,2.00170,248
3801,2.0,1599.318261,1601.319921,1.0,0.04,"[250.0, 250.0]",0.8,drifting_gratings,"[42471.86666667, 42471.86666667]",0.0,2.00166,249
3802,2.0,1602.320789,1604.322439,null,null,"[250.0, 250.0]",null,drifting_gratings,"[42471.86666667, 42471.86666667]",null,2.00165,250


In [11]:
stimuli_ids = stimuli_table.index

In [12]:
spikes_per_stimulus = oursession.presentationwise_spike_times(stimulus_presentation_ids = stimuli_ids)

In [13]:
spikes_per_stimulus.reset_index(inplace=True)

In [14]:
spike_counts = spikes_per_stimulus.groupby(['stimulus_presentation_id', 'unit_id'])['spike_time'].count().reset_index()
spike_counts.rename(columns={'spike_time': 'spike_count'}, inplace=True)
spike_counts

,stimulus_presentation_id,unit_id,spike_count
0,3798,951087020,20
1,3798,951087105,39
2,3798,951087261,11
3,3798,951087291,23
4,3798,951087330,2
...,...,...,...
321784,49432,951108207,8
321785,49432,951108231,6
321786,49432,951108474,11
321787,49432,951108512,3


In [18]:
dataset = pd.merge(spike_counts, stimuli_table[['spatial_frequency', 'temporal_frequency', 'duration', 'orientation']], on='stimulus_presentation_id')
dataset['spikes_per_second'] = dataset['spike_count'] / dataset['duration']
dataset = dataset.drop(columns=['duration', 'spike_count'])
dataset

,stimulus_presentation_id,unit_id,spatial_frequency,temporal_frequency,orientation,spikes_per_second
0,3798,951087020,0.04,4.0,135.0,9.991457
1,3798,951087105,0.04,4.0,135.0,19.483342
2,3798,951087261,0.04,4.0,135.0,5.495302
3,3798,951087291,0.04,4.0,135.0,11.490176
4,3798,951087330,0.04,4.0,135.0,0.999146
...,...,...,...,...,...,...
321784,49432,951108207,0.04,8.0,0.0,3.996663
321785,49432,951108231,0.04,8.0,0.0,2.997497
321786,49432,951108474,0.04,8.0,0.0,5.495411
321787,49432,951108512,0.04,8.0,0.0,1.498749


In [22]:
print(dataset['spatial_frequency'].value_counts())
#we drop the column spatial frequency because it is always 0.04
dataset = dataset.drop(columns=['spatial_frequency'])

0.04    306986
null     14803
Name: spatial_frequency, dtype: int64


In [32]:
#we add the area of the brain where the unit is located
units = cache.get_units()
dataset['area'] = units.loc[dataset['unit_id'], 'ecephys_structure_acronym'].values
dataset

,stimulus_presentation_id,unit_id,temporal_frequency,orientation,spikes_per_second,area
0,3798,951087020,4.0,135.0,9.991457,CA1
1,3798,951087105,4.0,135.0,19.483342,CA1
2,3798,951087261,4.0,135.0,5.495302,CA1
3,3798,951087291,4.0,135.0,11.490176,CA1
4,3798,951087330,4.0,135.0,0.999146,CA1
...,...,...,...,...,...,...
321784,49432,951108207,8.0,0.0,3.996663,VISrl
321785,49432,951108231,8.0,0.0,2.997497,VISrl
321786,49432,951108474,8.0,0.0,5.495411,CA1
321787,49432,951108512,8.0,0.0,1.498749,VISrl


In [34]:
dataset.to_csv(f'ourdata/dataset_session_{session}.csv', index=False)